# 데이터 파악

In [7]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [8]:
# 한글폰트 적용
import matplotlib.pyplot as plt
plt.rc("font",family="AppleGothic")

#마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus']=False

In [9]:
import pandas as pd
channel = pd.read_csv('./DATA/채널이용.txt', encoding='euc-kr', sep=',')
compuse = pd.read_csv('./DATA/경쟁사이용.txt', encoding='euc-kr', sep=',')
prodcat = pd.read_csv('./DATA/상품분류.txt', encoding='euc-kr', sep=',')
purprd1 = pd.read_csv('./DATA/구매상품TR.txt', encoding='euc-kr', sep=',')
membership = pd.read_csv('./DATA/멤버십여부.txt', encoding='euc-kr', sep=',')
cust = pd.read_csv('./DATA/고객DEMO.txt', encoding='euc-kr', sep=',')


FileNotFoundError: [Errno 2] No such file or directory: './DATA/채널이용.txt'

## 제외 데이터
1. channel: 채널이용 데이터
2. compuse: 경쟁사 이용현황 데이터

In [ ]:
# 1. channel: 채널이용 데이터
# 채널 이용의 기준일시를 알 수 없으므로 모델에 적용하기 어려움
channel.head(3)

In [ ]:
# 2. compuse: 경쟁사 이용현황 데이터
# 데이터의 이용년월의 최소값이 2015년 1월이므로 학습 데이터에 사용할 수 없음
print(compuse['이용년월'].min())
compuse.head(3)

## 사용 데이터
1. procl: 제휴사별 판매물품의 대/중/소분류 및 중/소분류명 정보 데이터
2. purprd: 고객별 영수증 데이터
3. membership: 고객별 멤버십 가입 데이터
4. cust: 고객별 인구 통계학적 정보 데이터


In [ ]:
# 1. procl: 제휴사별 판매물품의 대/중/소분류 및 중/소분류명 정보 데이터
prodcat.head(3)

In [ ]:
# 제휴사별 대/중/소 분류 체계가 다름
# 대분류코드 일원화 실시. 대분류코드보다 상위 개념인 대대분류코드도 새로 만들어 purprd에 적용함

for i in ['A','B','C','D']:
    counts = prodcat[(prodcat['제휴사']==i)]['대분류코드'].nunique(),\
            prodcat[(prodcat['제휴사']==i)]['중분류코드'].nunique(),\
            prodcat[(prodcat['제휴사']==i)]['소분류코드'].nunique()
    print(f'제휴사 {i}의 대/중/소 분류코드 수:', counts)

In [ ]:
# 2. purprd: 고객별 영수증 데이터
# prodcat의 카테고리 분류체계 변경을 반영함
# 대분류코드, 중분류코드, 소분류코드 -> 대대분류코드, 새대분류코드, 중분류코드, 소분류코드
purprd1.head(3)

In [ ]:
procl = prodcat.copy()

In [ ]:
#가공식품
procl.loc[procl['중분류명'].str.contains(r"규격RTC|식사류|건면|즉석식품|발효유|식용유|만두|쿠키|초콜렛|씨리얼|케첩마요네즈|냉장식사|축산가공|가공식품|주류|커피|대용식|음식조리|도시락|빵|HMR|밥죽류|간편요리|냉장간편식|간식|선식|햄|소시지|아이스크림|생수|유제품|요구르트|브랜드빵|치즈|두유|음료|소주|양주|건강식품|담배|꿀|안주|한과|프리미엄과자|과자|피자|디저트|장류|식용유지|가루|수입조미|면류|분유|잼|통조림|이유식|구이찜|건명태|튀김|밥류|선물세트|즉석구이|향신료|설탕류|조리식품|라면|인스턴트|간편조리|맥주|와인|전통주|스낵|껌|캔디|시리얼|파이|비스켓|초콜릿|양념|식자재|마른안주|커피|차|소스|조미료|냉동|가공|육포"),'중분류명'] = '가공식품'
#외식
procl.loc[procl['중분류명'].str.contains(r"맛집행사|컨세션|푸드몰|푸드코트|먹거리|중식|패스트푸드|분식|일식|양식|한식|레스토랑|Fast|카페|편의시설|판매시설|주유소|브랜드조리|베이커리|이벤트조리"),'중분류명'] = '외식'
#일상용품
procl.loc[procl['중분류명'].str.contains(r"음식저장|조리용기|세탁세제|화장지|주방주거세제|화장품|두발용품|생리용품|구강용품|위생용품|주방용품|청소욕실용품|가정잡화|화장지|탈취제|베이직케어|선케어|클렌징|집중케어|남성케어|베이비케어|헤어케어|바디케어|풋케어|구강케어|여성용품|화장지티슈|방향제습탈취제|미용소품|안전용품|여행용품|계절팩|여성|남성|베이스메이크업|아이메이크업|립메이크업|네일메이크업|주방용품|생활잡화|식기|일용잡화|화장품조리용기|조리도구|미용소품|훼이셜케어|공구|종량제봉투|필기도구|미용잡화|노트|드럭스토어|아로마|핸드케어|덴탈케어|화장품|건전지|용품|주방|휴지통|밀대|바디|스킨케어|욕실|헤어|일상용품|티슈|청소편의|세제|제습제|키친타올|위생용품|생리대|섬유유연제|표백제|살충제|탈취제|기저귀|화장지"),'중분류명'] = '일상용품'
#의약품 의료기기 
procl.loc[procl['중분류명'].str.contains(r"건강기능식품|일반의약외품|건강식품|홍인삼|기능성건강|비타민"),'중분류명'] = '의약품/의료기기'
#교육 문화
procl.loc[procl['중분류명'].str.contains(r"문화용품|악기|팬시|교육|EDUCATION|서적|음반"),'중분류명'] = '교육/문화용품'
#디지털가전
procl.loc[procl['중분류명'].str.contains(r"대형가전|소형가전|컴퓨터|전자|주방가전|VIDEO|에어컨|TV|DVD|오디오|홈시어터|정수기|공기청정기|음향기기|히터|전기요|가습기|밥솥|청소기|전기포트|믹서|조리기|소형음향기기|카메라|전화기|이동통신|PC|컴퓨터|전기면도기|하이패스|전자악기|MP3|전자학습기|냉장고|세탁기|선풍기|가전|디지털"),'중분류명'] = '디지털/가전'
procl.loc[procl['소분류명'].str.contains(r"헤어드라이어|헤어세팅기"),'중분류명'] = '디지털/가전'

#가구
procl.loc[procl['중분류명'].str.contains(r"수예|매트|전구|그릇|침구|커튼|수예|조명|가구|테이블|의자|퍼니처|벽지"),'중분류명'] = '가구/인테리어'
procl.loc[procl['소분류명'].str.contains(r"교자상|다용도상"),'중분류명'] = '가구/인테리어'

#스포츠
procl.loc[procl['중분류명'].str.contains(r"레저취미|SPORTS|스키|등산|낚시|수영복|롤러보드|자전거|골프|스포츠"),'중분류명'] = '전문스포츠/레저'
#패션잡화
procl.loc[procl['중분류명'].str.contains(r"아동슈즈|양산|피트니스|액세서리|피혁잡화|구두|복합샵|시즌잡화|슈즈|스타킹|드레스화브랜드|슈즈행사|샌들슬리퍼|NB구두|운동화|준보석|시계|액세서리|피혁브랜드|가방브랜드|잡화멀티샵|미입점행사|지갑|벨트|핸드백|가방|피혁토탈|모자|스카프|장갑|시즌토탈|썬글라스|양말|우산"),'중분류명'] = '패션잡화'
procl.loc[procl['소분류명'].str.contains(r"선글라스|피혁"),'중분류명'] = '패션잡화'

#의류
procl.loc[procl['중분류명'].str.contains(r"섬유잡화|캐주얼|커리어|트래디셔널|시티웨어|내의|블라우스|기타의류|디자이너|모피|피혁|남성정장|남성캐주얼|교복|셔츠|트랜디|병행수입|수입브랜드|마담|덧신|내의|란제리|웨어|주니어|의류|캐주얼|스트리트|정장|남성|남아|여성|여아|아동"),'중분류명'] = '의류'
#유아용품
procl.loc[procl['중분류명'].str.contains(r"아동|GIRLS|BOYS|베이비|BABY|유아"),'중분류명'] = '유아용품'
#명품
procl.loc[procl['중분류명'].str.contains(r"보석|준보석|명품"),'중분류명'] = '명품'
#기타
procl.loc[procl['중분류명'].str.contains(r"브랜드|기타|웨딩|데일리처분|균일가|파티|시즌행사|영상|동물병원|카시트|캠핑|여행|TRAVEL"),'중분류명'] = '기타'
procl.loc[procl['소분류명'].str.contains(r'일반브랜드'), '중분류명'] = '기타'

#신선식품
procl.loc[procl['중분류명'].str.contains(r"특산물|견과류|냉장농산|냉장수산|냉장축산|냉장기타|채류|돈육|계육|건어류|해물|육류|농산물|수산품|반찬|단무지|잡곡|초밥|다시마|수입게|떡|족발|순대|연체|새우|마른김|조미김|연어|갈치|굴비|조개|계란|오리|우육|닭|감귤|감|복숭아|자두|구이|토마토|기타회|건과|생활한방|한방식재류|수입육|한우|포도|참외|수박|딸기|메론|오렌지|자몽|석류|레몬|해물연체류|블루베리|체리|수입포도|바나나|파인애플|망고|키위|사과|배|밤|과일|해초|나물|두부|묵|즉석참기름|소고기|돼지고기|김치|반찬|양곡|멸치|황태|우유|즉석도정미|특산물|국산게|해조류|꽁치|조기|고등어|약초|대구|도미|생선|잎|잡곡류|선어|채소|버섯|오징어|젓갈|쥐치포|육포|건명태"),'중분류명'] = '신선식품'

procl

In [ ]:
# 중분류분류코드 컬럼 추가
procl['중분류분류코드'] = procl['중분류명'].map({'신선식품':12,'가공식품':11,'외식':13,'일상용품':14,'의약품/의료기기':15,'교육/문화용품':16,'기타':17,'디지털/가전':21,'가구/인테리어':22,'의류':23,'전문스포츠/레저':24,'패션잡화':25,'유아용품':26,'명품':31})
procl.head(3)

In [ ]:
procl.중분류분류코드.unique()
procl.중분류명.unique()

In [ ]:
procl.중분류명.unique()

In [ ]:
# 내구재 카테고리 나누기
def naegujae(x):
    cat=''
    if x in ['가구/인테리어','디지털/가전','명품']:
        cat = '100'
    elif x in ['의류','전문스포츠/레저','패션잡화','일상용품']:
        cat = '200'
    elif x in ['가공식품','교육/문화용품','신선식품','의약품/의료기기','유아용품']:
        cat = '300'      
    elif x in ['외식','기타']:
        cat = '400'
    return cat


procl['내구재분류코드'] = procl['중분류명'].map(naegujae)

In [ ]:
# 소비재 카테고리 나누기
def sobijae(x):
    cat=''
    if x in ['가공식품','신선식품','외식','일상용품','의약품/의료기기','교육/문화용품','기타']:
        cat = '1'
    elif x in ['디지털/가전','가구/인테리어','의류','전문스포츠/레저','패션잡화','유아용품']:
        cat = '2'
    elif x =='명품':
        cat = '3'
        
    return cat

procl['소비재분류코드'] = procl['중분류명'].map(sobijae)

In [ ]:
procl.head(3)

In [ ]:
display(procl.head(3),purprd1.head(3))

In [ ]:
procl= procl.drop(['제휴사','중분류코드', '대분류코드'] , axis=1)
purprd = pd.merge(purprd1,procl,on='소분류코드',how='inner')
purprd.head(3)

In [ ]:
for i in ['A','B','C','D']:
    counts = purprd[(purprd['제휴사']==i)]['소비재분류코드'].nunique(),\
            purprd[(purprd['제휴사']==i)]['내구재분류코드'].nunique(),\
            purprd[(purprd['제휴사']==i)]['중분류분류코드'].nunique()
    print(f'제휴사 {i}의 소비재/내구재/중분류 분류코드 수:', counts)

In [ ]:
pd.pivot_table(data=purprd, columns='제휴사', index='소비재분류코드', aggfunc='size')

In [ ]:
pd.pivot_table(data=purprd, columns='제휴사', index='내구재분류코드', aggfunc='size')

In [ ]:
pd.pivot_table(data=purprd, columns='제휴사', index='중분류분류코드', aggfunc='size')

In [ ]:
purprd.columns

In [ ]:
purprd = purprd[['제휴사', '영수증번호', '대분류코드', '중분류코드', '소분류코드', '소비재분류코드', '내구재분류코드', '중분류분류코드','중분류명', '소분류명','고객번호', '점포코드', '구매일자',
       '구매시간', '구매금액']]

In [ ]:
purprd.head(3)

In [ ]:
# 3. membership: 고객별 멤버십 가입 데이터
membership.head(3)

In [ ]:
# 4. cust: 고객별 인구 통계학적 정보 데이터
cust.head(3)

# 반기별 고객구매 df 만들기
- cust 데이터 기반, 2014년 1/2반기, 2015년 1/2반기의 고객별 구매정보 데이터 프레임 형성

In [ ]:
# purprd 수정

# 1. 구매일자 datetime 형식으로 변환
purprd.구매일자 = pd.to_datetime(purprd.구매일자, format = '%Y%m%d')
purprd['구매요일'] = purprd['구매일자'].dt.day_name()
# 2. 구매시간 범주화

def time_cat(x):
    if 9 <= x < 12: return '오전'
    elif 12 <= x < 15: return '낮'
    elif 15 <= x < 18: return '오후'
    elif 18 <= x < 21: return '저녁'
    else: return '야간'    
purprd.구매시간 = purprd.구매시간.apply(time_cat)

# 평일(1)/주말(0) 분류
# 주중/주말 카테고리 나누기

def week_cat(x):
    if x in ['Saturday','Sunday']:
        cat = 0
    else:
        cat = 1
    return cat

purprd['평일주말'] = purprd['구매요일'].map(week_cat)

# 3. 점포코드를 제휴사와 합친 형태로 변환 (ex. 제휴사 B, 점포코드 101 -> 점포코드 B101)
purprd.점포코드 = purprd.제휴사 + purprd.점포코드.astype(str)

# 4. purprd 반기별 분할
purprd_14_half1 = purprd[(purprd.구매일자>=pd.to_datetime(20140101, format='%Y%m%d'))&(purprd.구매일자<pd.to_datetime(20140701, format='%Y%m%d'))]
purprd_14_half2 = purprd[(purprd.구매일자>=pd.to_datetime(20140701, format='%Y%m%d'))&(purprd.구매일자<pd.to_datetime(20150101, format='%Y%m%d'))]
purprd_15_half1 = purprd[(purprd.구매일자>=pd.to_datetime(20150101, format='%Y%m%d'))&(purprd.구매일자<pd.to_datetime(20150701, format='%Y%m%d'))]
purprd_15_half2 = purprd[(purprd.구매일자>=pd.to_datetime(20150701, format='%Y%m%d'))&(purprd.구매일자<pd.to_datetime(20160101, format='%Y%m%d'))]
purprd_14_half1.head(3)

In [ ]:
# cust에 membership 가입년월 컬럼 추가
# 여러가지 멤버십 복수가입 시, 가장 일찍 가입한 멤버십 기준으로 가입년월 집계함(min)
# 가입여부가 아닌 가입년월인 이유: 반기별로 고객데이터 분할한 이후에 가입여부가 바뀔 수 있기 때문
# 반기별 고객데이터 분할 이후 가입년월을 가입여부로 변경

membership.가입년월 = pd.to_datetime(membership.가입년월, format = '%Y%m')
cust = cust.merge(membership.pivot_table(index = '고객번호', values = '가입년월', aggfunc='min'), how = 'left', on = '고객번호').rename({'가입년월':'멤버십가입년월'}, axis = 1)
cust.head(3)

In [ ]:
# 연령대 카테고리 나누기
# 학생층:0,청년층:1,중년층:2,장년층:3,노년층:4

def age_cat(x):
    if x in ['19세이하','20세~24세']:
        cat = 0
    elif x in ['25세~29세','30세~34세']:
        cat = 1
    elif  x in ['35세~39세','40세~44세']:
        cat = 2      
    elif  x in ['45세~49세','50세~54세']:
        cat = 3
    else:
        cat = 4
    return cat


cust['연령대'] = cust['연령대'].map(age_cat)

In [ ]:
# 반기별 고객구매 df 만들기: cust를 반기별로 복사

cust_14_half1 = cust.copy()
cust_14_half2 = cust.copy()
cust_15_half1 = cust.copy()
cust_15_half2 = cust.copy()

In [ ]:
# 변수수정: 멤버십가입년월 -> 멤버십가입여부

ref_date = pd.to_datetime(20140701, format = '%Y%m%d')
cust_14_half1.멤버십가입년월 = cust_14_half1.멤버십가입년월.map(lambda x: 1 if x < ref_date else 0)
ref_date = pd.to_datetime(20150101, format = '%Y%m%d')
cust_14_half2.멤버십가입년월 = cust_14_half2.멤버십가입년월.map(lambda x: 1 if x < ref_date else 0)
ref_date = pd.to_datetime(20150701, format = '%Y%m%d')
cust_15_half1.멤버십가입년월 = cust_15_half1.멤버십가입년월.map(lambda x: 1 if x < ref_date else 0)
ref_date = pd.to_datetime(20160101, format = '%Y%m%d')
cust_15_half2.멤버십가입년월 = cust_15_half2.멤버십가입년월.map(lambda x: 1 if x < ref_date else 0)

cust_14_half1 = cust_14_half1.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)
cust_14_half2 = cust_14_half2.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)
cust_15_half1 = cust_15_half1.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)
cust_15_half2 = cust_15_half2.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)

cust_14_half1.head(3)

# 파생변수 생성

## 1. 제휴사별 구매금액비율

In [ ]:
def get_amount_rate(purprd, cust):
    temp = purprd.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0)
    temp['총계'] = temp.sum(axis = 1)
    temp = temp.apply(lambda x: x/temp['총계'])
    temp = temp.drop('총계', axis = 1)
    cust = cust.merge(temp, how = 'left', on = '고객번호').rename({'A':'A구매금액비율', 'B':'B구매금액비율', 'C':'C구매금액비율', 'D':'D구매금액비율'}, axis = 1)
    return cust

cust_14_half1 = get_amount_rate(purprd_14_half1,cust_14_half1)
cust_14_half2 = get_amount_rate(purprd_14_half2,cust_14_half2)
cust_15_half1 = get_amount_rate(purprd_15_half1,cust_15_half1)
cust_15_half2 = get_amount_rate(purprd_15_half2,cust_15_half2)

cust_14_half1.head(3)

## 2. 주요구매시간대

In [ ]:
import numpy as np
from scipy.stats import mode

def get_feq_time(purprd, cust):
    temp = purprd.groupby(['고객번호', '영수증번호']).구매시간.first().reset_index().groupby('고객번호').구매시간.apply(lambda x: mode(x)[0][0])
    cust = cust.merge(temp.rename('주요구매시간대'), how = 'left', on = '고객번호')
    return cust

cust_14_half1 = get_feq_time(purprd_14_half1,cust_14_half1)
cust_14_half2 = get_feq_time(purprd_14_half2,cust_14_half2)
cust_15_half1 = get_feq_time(purprd_15_half1,cust_15_half1)
cust_15_half2 = get_feq_time(purprd_15_half2,cust_15_half2)

cust_14_half1.head(3)

## 3. 주중/주말별

In [ ]:
import numpy as np
from scipy.stats import mode

def get_feq_holiday(purprd, cust):
    temp = purprd.groupby(['고객번호', '영수증번호']).평일주말.first().reset_index().groupby('고객번호').평일주말.apply(lambda x: mode(x)[0][0])
    cust = cust.merge(temp.rename('평일주말별'), how = 'left', on = '고객번호')
    return cust

cust_14_half1 = get_feq_holiday(purprd_14_half1,cust_14_half1)
cust_14_half2 = get_feq_holiday(purprd_14_half2,cust_14_half2)
cust_15_half1 = get_feq_holiday(purprd_15_half1,cust_15_half1)
cust_15_half2 = get_feq_holiday(purprd_15_half2,cust_15_half2)

cust_14_half1.head(3)

In [ ]:
purprd[:3]

In [ ]:
cust[:3]

## 4. 연령대별(보류)

def get_amount_age(purprd, cust):
    temp = purprd.pivot_table(index = '고객번호', columns = '연령대', values = '구매금액', aggfunc = 'sum').fillna(0)
    temp['총계'] = temp.sum(axis = 1)
    temp = temp.apply(lambda x: x/temp['총계'])
    temp = temp.drop('총계', axis = 1)
    cust = cust.merge(temp, how = 'left', on = '고객번호').rename({'A':'A구매금액비율', 'B':'B구매금액비율', 'C':'C구매금액비율', 'D':'D구매금액비율'}, axis = 1)
    return cust

cust_14_half1 = get_amount_age(purprd_14_half1,cust_14_half1)
cust_14_half2 = get_amount_age(purprd_14_half2,cust_14_half2)
cust_15_half1 = get_amount_age(purprd_15_half1,cust_15_half1)
cust_15_half2 = get_amount_age(purprd_15_half2,cust_15_half2)

cust_14_half1.head(3)

## 5. 제휴사별 방문횟수


In [ ]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

## 6. 제휴사별 상품구매개수

In [ ]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

## 7. 총구매금액


In [ ]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')

# 2014-1반기 기준, 각 반기의 매출 증감에 대해 보정
r2 = cust_14_half1.총구매금액.sum() / cust_14_half2.총구매금액.sum()
r3 = cust_14_half1.총구매금액.sum() / cust_15_half1.총구매금액.sum()
r4 = cust_14_half1.총구매금액.sum() / cust_15_half2.총구매금액.sum()

cust_14_half2.총구매금액 = cust_14_half2.총구매금액*r2
cust_15_half1.총구매금액 = cust_15_half1.총구매금액*r3
cust_15_half2.총구매금액 = cust_15_half2.총구매금액*r4

cust_14_half1.head(3)

## 8. 방문점포종류수

In [ ]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')

cust_14_half1.head(3)

## 9. 마지막구매일자

In [ ]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.마지막구매일자 = cust_14_half1.마지막구매일자.astype('datetime64')
cust_14_half2.마지막구매일자 = cust_14_half2.마지막구매일자.astype('datetime64')
cust_15_half1.마지막구매일자 = cust_15_half1.마지막구매일자.astype('datetime64')
cust_15_half2.마지막구매일자 = cust_15_half2.마지막구매일자.astype('datetime64')

cust_14_half1.head(3)

## 10. 회당구매금액

In [ ]:
cust_14_half1['회당구매금액'] = cust_14_half1.총구매금액/(cust_14_half1.A방문횟수 + cust_14_half1.B방문횟수 + cust_14_half1.C방문횟수 + cust_14_half1.D방문횟수)
cust_14_half2['회당구매금액'] = cust_14_half2.총구매금액/(cust_14_half2.A방문횟수 + cust_14_half2.B방문횟수 + cust_14_half2.C방문횟수 + cust_14_half2.D방문횟수)
cust_15_half1['회당구매금액'] = cust_15_half1.총구매금액/(cust_15_half1.A방문횟수 + cust_15_half1.B방문횟수 + cust_15_half1.C방문횟수 + cust_15_half1.D방문횟수)
cust_15_half2['회당구매금액'] = cust_15_half2.총구매금액/(cust_15_half2.A방문횟수 + cust_15_half2.B방문횟수 + cust_15_half2.C방문횟수 + cust_15_half2.D방문횟수)

cust_14_half1.head(3)

## 11. 구매상품종류수
- 소분류 기준

In [ ]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'소분류구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'소분류구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'소분류구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'소분류구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

- 내구재분류 기준

In [ ]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', values = '내구재분류코드', aggfunc = 'nunique').rename({'내구재분류코드':'내구재구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', values = '내구재분류코드', aggfunc = 'nunique').rename({'내구재분류코드':'내구재구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', values = '내구재분류코드', aggfunc = 'nunique').rename({'내구재분류코드':'내구재구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', values = '내구재분류코드', aggfunc = 'nunique').rename({'내구재분류코드':'내구재구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

- 소비재분류 기준

In [ ]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', values = '소비재분류코드', aggfunc = 'nunique').rename({'소비재분류코드':'소비재구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', values = '소비재분류코드', aggfunc = 'nunique').rename({'소비재분류코드':'소비재구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', values = '소비재분류코드', aggfunc = 'nunique').rename({'소비재분류코드':'소비재구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', values = '소비재분류코드', aggfunc = 'nunique').rename({'소비재분류코드':'소비재구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

- 중분류분류 기준

In [ ]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', values = '중분류분류코드', aggfunc = 'nunique').rename({'중분류분류코드':'중분류구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', values = '중분류분류코드', aggfunc = 'nunique').rename({'중분류분류코드':'중분류구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', values = '중분류분류코드', aggfunc = 'nunique').rename({'중분류분류코드':'중분류구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', values = '중분류분류코드', aggfunc = 'nunique').rename({'중분류분류코드':'중분류구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

## 12. 제휴사별 구매금액

In [ ]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

## 13. 중뷴류분류코드별 구매순위 등급
- 4등급으로 분류

In [ ]:
# 분기별 구매데이터프레임 분할(총 8분기)
# fh: first half(상반기), sh: second half(하반기)

fh14_1 = purprd[purprd.구매일자<'20140401']
fh14_2 = purprd[(purprd.구매일자>='20140401')&(purprd.구매일자<'20140701')]
sh14_1 = purprd[(purprd.구매일자>='20140701')&(purprd.구매일자<'20141001')]
sh14_2 = purprd[(purprd.구매일자>='20141001')&(purprd.구매일자<'20150101')]

fh15_1 = purprd[(purprd.구매일자>='20150101')&(purprd.구매일자<'20150401')]
fh15_2 = purprd[(purprd.구매일자>='20150401')&(purprd.구매일자<'20150701')]
sh15_1 = purprd[(purprd.구매일자>='20150701')&(purprd.구매일자<'20151001')]
sh15_2 = purprd[(purprd.구매일자>='20151001')&(purprd.구매일자<'20160101')]


# 고객별 중분류분류별 구매금액 총액에 대해 순위 매기기(오름차순)
# 카테고리 구매가 없는 고객은 모두 1위로 지정

fh14_1_pv = fh14_1.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
fh14_2_pv = fh14_2.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
sh14_1_pv = sh14_1.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
sh14_2_pv = sh14_2.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)

fh15_1_pv = fh15_1.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
fh15_2_pv = fh15_2.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
sh15_1_pv = sh15_1.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
sh15_2_pv = sh15_2.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)

# 순위에 따라 4등급으로 분류하는 함수 작성(총 고객수의 4분위수 기준으로 분류)
def rank(x):
    if x <= 4846: return 1
    elif x <= 9692: return 2
    elif x <= 14539: return 3
    else: return 4

In [ ]:
# 전체 8분기, 각 중분류 12개에 대한 등급 매기기
for el in [fh14_1_pv, fh14_2_pv, sh14_1_pv, sh14_2_pv, fh15_1_pv, fh15_2_pv, sh15_1_pv, sh15_2_pv]:
    if i in el:
        for i in range(1,13):
            el[i] = el[i].apply(lambda x:rank(x))
    else:
        pass

# 반기별 데이터프레임에 합병
cust_14_half1 = cust_14_half1.merge(fh14_1_pv, on='고객번호')
cust_14_half1 = cust_14_half1.merge(fh14_2_pv, on='고객번호', suffixes=['_1','_2'])
cust_14_half2 = cust_14_half2.merge(sh14_1_pv, on='고객번호')
cust_14_half2 = cust_14_half2.merge(sh14_2_pv, on='고객번호', suffixes=['_1','_2'])

cust_15_half1 = cust_15_half1.merge(fh15_1_pv, on='고객번호')
cust_15_half1 = cust_15_half1.merge(fh15_2_pv, on='고객번호', suffixes=['_1','_2'])
cust_15_half2 = cust_15_half2.merge(sh15_1_pv, on='고객번호')
cust_15_half2 = cust_15_half2.merge(sh15_2_pv, on='고객번호', suffixes=['_1','_2'])

cust_14_half1.head(3)

## 14. 중뷴류분류코드별 구매금액

In [ ]:
cust_category_14_half1 = purprd_14_half1.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().fillna(0)
cust_category_14_half2 = purprd_14_half2.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().fillna(0)
cust_category_15_half1 = purprd_15_half1.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().fillna(0)
cust_category_15_half2 = purprd_15_half2.groupby(['고객번호','중분류분류코드'])['구매금액'].sum().unstack().fillna(0)

cust_14_half1 = cust_14_half1.merge(cust_category_14_half1, on='고객번호', how='left')
cust_14_half2 = cust_14_half2.merge(cust_category_14_half2, on='고객번호', how='left')
cust_15_half1 = cust_15_half1.merge(cust_category_15_half1, on='고객번호', how='left')
cust_15_half2 = cust_15_half2.merge(cust_category_15_half2, on='고객번호', how='left')

cust_14_half1.head(3)

In [ ]:
# 고객별, 분기별 구매 데이터 저장

cust_14_half1.to_csv('DATA/cust_14_half1.csv', encoding='utf-8', index=False)
cust_14_half2.to_csv('DATA/cust_14_half2.csv', encoding='utf-8', index=False)
cust_15_half1.to_csv('DATA/cust_15_half1.csv', encoding='utf-8', index=False)
cust_15_half2.to_csv('DATA/cust_15_half2.csv', encoding='utf-8', index=False)

In [ ]:
cust_14_half1.columns